# code to chunk ouput into tokens and check if the output is correct


In [1]:
problem2="""
Solve the following mathematical problem: The manager of a company planned to distribute a $$50$ bonus to each employee from the company fund, but the fund contained $$5$ less than what was needed.  Instead the manager gave each employee a $$45$ bonus and kept the remaining $$95$ in the company fund.  The amount of money in the company fund before any bonuses were paid was
$\text{(A)}\ 945\text{ dollars} \qquad \text{(B)}\ 950\text{ dollars} \qquad \text{(C)}\ 955\text{ dollars} \qquad \text{(D)}\ 990\text{ dollars} \qquad \text{(E)}\ 995\text{ dollars}$
### Solution: To solve this problem, let's define the variables and set up the equations based on the given information.

1. Let \( n \) be the number of employees 1.2 A.
2. The manager initially planned to distribute $50 to each employee, so the total amount needed was \( 50n \). However, the fund was $5 less than what was needed, so the initial fund amount was \( 50n - 5 \).
3. Instead, the manager distributed $45 to each employee, which amounted to \( 45n \), and kept the remaining $95 in the fund. Therefore, the initial fund amount was \( 45n + 95 \).

We can set up the following equation:
\[ 50n - 5 = 45n + 95 \]

Let's solve this equation using Python and sympy.

```python
import sympy as sp

# Define the variable
n = sp.symbols('n')

# Set up the equation
equation = sp.Eq(50*n - 5, 45*n + 95)

# Solve the equation for n
solution = sp.solve(equation, n)

# Calculate the initial fund amount
initial_fund = 50 * solution[0] - 5
print(initial_fund)
```
```output
995
```
The amount of money in the company fund before any bonuses were paid was \(\boxed{995}\) dollars.
"""

In [34]:
import re

# Use regex to dynamically identify and extract steps and other parts of the problem
def split_problem(problem_text):
    # Regex patterns to match steps
    step_pattern = r'(\d+\.\s[A-Z].*?)(?=\d+\.\s[A-Z]|\n\n)'

    # Extract the prestep from "Solution:" to the first double line break
    pre_step_pattern = r'(### Solution:.*?)(?=\n\n)'
    pre_step_match = re.search(pre_step_pattern, problem_text, re.DOTALL)
    pre_step = pre_step_match.group(0).strip() if pre_step_match else ""

    # Extract all steps dynamically
    steps = re.findall(step_pattern, problem_text, re.DOTALL)

    # If there are steps, get the remaining part after the last step
    if steps:
        # Find the last step's position
        last_step = steps[-1]
        last_step_end = problem_text.rfind(last_step) + len(last_step)
        # Everything after the last step is considered the final result
        final_result = problem_text[last_step_end:].strip()
    else:
        final_result = ""

    # Combine all parts into a list
    combined_result = [pre_step] + steps + [final_result]
    
    return combined_result

# Example problem text
problem = """
A teacher tells the class, Ben thinks of 6 , and gives his answer to Sue.  
What should Sue's answer be?

### Solution: To solve this problem, we need to determine what Sue should get when she receives Ben's answer. 
The problem states that Ben thinks of 6, and we need to find the corresponding value that Sue should get.

Let's break down the problem step-by-step:

1. Ben thinks of the number 1.6 A.
2. The teacher tells Sue to add 15 to Ben's number.
3. Sue then gives her answer to another student.

Let's use Python and sympy to solve this problem:

import sympy as sp

# Define the number Ben thinks of
ben_number = 6

# Define the operation Sue should perform
operation_result = ben_number + 15

# Print the result
print(operation_result)
"""

# # Call the function to split the problem text dynamically
# result_list = split_problem(problem2)

# # Print the list
# for i, item in enumerate(result_list):
#     print(f"Part {i + 1}: {item.strip()}")


In [35]:
!git clone https://huggingface.co/Pra-tham/BART_validator
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


fatal: destination path 'BART_validator' already exists and is not an empty directory.


In [36]:
import torch
from transformers import BartForSequenceClassification, BartTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding


class BartForRegression(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bart = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels=1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bart(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = None
        if labels is not None:
            loss_fct = torch.nn.MSELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
        return {'loss': loss, 'logits': logits}

# Instantiate the model


In [37]:
model_path = "/kaggle/working/BART_validator/pytorch_model.bin"
config_path = "/kaggle/working/BART_validator/config.json"
from transformers import BartTokenizer
# Initialize the model and load the state dictionary
bart_model = BartForRegression()
bart_model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# Load the tokenizer from Hugging Face Hub
# tokenizer = AutoTokenizer.from_pretrained("Pra-tham/BART_validator")

# Step 2: Prepare input data
text = "This is an example sentence for BART regression inference."

def validation_value(text):
    inputs = bart_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    bart_model.to(device)

    # Step 2: Prepare input data
#     text = "This is an example sentence for BART regression inference."
#     inputs = bart_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # Step 3: Make a prediction (inference)
    bart_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        bart_outputs = bart_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = bart_outputs['logits']
        return logits

# Output logits
print("Logits (regression output):", validation_value(text))

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31/2768746558.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this 

Logits (regression output): tensor([[0.9407]], device='cuda:0')


# all integrated

In [38]:
!pip install ctranslate2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
import os
import re
import subprocess
import tempfile
import multiprocessing
from collections import Counter
from contextlib import contextmanager
from dataclasses import dataclass


class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    @staticmethod
    def _run_code(temp_file_path):
        result = subprocess.run(
            ["python3", temp_file_path],
            capture_output=True,
            check=False,
            text=True
        )
        if result.returncode == 0:
            return True, result.stdout.strip()
        else:
            error_msg = result.stderr.strip()
            msgs = error_msg.split("\n")
            new_msgs = []
            want_next = False
            for m in msgs:
                if "Traceback" in m:
                    new_msgs.append(m)
                elif m == msgs[-1]:
                    new_msgs.append(m)
                elif temp_file_path in m:
                    st = m.index('"/') + 1 if '"/' in m else 0
                    ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                    clr = m[st:ed] if not ed else m[st:]
                    m = m.replace(clr, "")
                    new_msgs.append(m)
                    want_next = True
                elif want_next:
                    new_msgs.append(m)
                    want_next = False
            return False, "\n".join(new_msgs).strip()

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)

            with multiprocessing.Pool(1) as pool:
                result = pool.apply_async(self._run_code, (temp_file_path,))
                try:
                    success, output = result.get(self.timeout)
                except multiprocessing.TimeoutError:
                    pool.terminate()
                    return False, f"Timed out after {self.timeout} seconds."
        return success, output


def execute_completion(executor, completion, return_status, last_code_block):
    executions = re.findall(r"```python(.*?)```", completion, re.DOTALL)
    if len(executions) == 0:
        return completion, False if return_status else completion
    if last_code_block:
        executions = [executions[-1]]
    outputs = []
    successes = []
    for code in executions:
        success = False
        for lib in ("subprocess", "venv"):
            if lib in code:
                output = f"{lib} is not allowed"
                outputs.append(output)
                successes.append(success)
                continue
        try:
            success, output = executor(code)
        except TimeoutError as e:
            print("Code timed out")
            output = e
        if not success and not return_status:
            output = ""
        outputs.append(output)
        successes.append(success)
    output = str(outputs[-1]).strip()
    success = successes[-1]
    if return_status:
        return output, success
    return output


def postprocess_completion(text, return_status, last_code_block):
    executor = PythonREPL()
    result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
    del executor
    return result


def get_majority_vote(answers):
    if not len(answers):
        return 0
    c = Counter(answers)
    value, freq = c.most_common(1)[0] 
    return value,freq


In [40]:


import ctranslate2
from transformers import AutoTokenizer
from huggingface_hub import snapshot_download


# Define the model and tokenizer loading
model_prompt = "Solve the following mathematical problem: "
tokenizer = AutoTokenizer.from_pretrained("AI-MO/NuminaMath-7B-TIR")
model_path = snapshot_download(repo_id="Makima57/deepseek-math-Numina")
# generator = ctranslate2.Generator(
#     model_path,         # Set device to GPU
#     device="cpu",
#     compute_type="int8",    # Use int8 quantization for lower memory usage
# #     device_index=1,         # (Optional) Specify the GPU device index (0 for first GPU)
#     # (Optional) Set memory utilization to low
# )

generator = ctranslate2.Generator(
    model_path, 
    device="cuda",          # Set device to GPU
    compute_type="int8",    # Use int8 quantization for lower memory usage
    device_index=1
)
iterations=10


# Function to generate predictions using the model
def get_prediction(question):
    input_text = model_prompt + question
    input_tokens = tokenizer.tokenize(input_text)
    results = generator.generate_batch([input_tokens],max_length=4096)
    
    output_tokens = results[0].sequences[0]
    predicted_answer = tokenizer.convert_tokens_to_string(output_tokens)
    return predicted_answer

# Function to perform majority voting across multiple predictions
def majority_vote(question, num_iterations=5):
    all_predictions = []
    all_answer=[]
    for _ in range(num_iterations):
        prediction = get_prediction(question)
        answer=postprocess_completion(prediction,True,True)
        all_predictions.append(prediction)
        all_answer.append(answer)
    majority_voted_pred = max(set(all_predictions), key=all_predictions.count)
    majority_voted_ans,freq=get_majority_vote(all_answer)
    print("frequency is ",freq)

    return majority_voted_pred, all_predictions,majority_voted_ans,freq


def new_input(chunks,solution_val_values):
        valid_chunks = []

        # Loop over the validation values and chunks together
        for chunk_idx, val_value in enumerate(solution_val_values):
            # Check if validation value is less than 0.9
            if val_value.item() < 0.9:
                print(f"Stopping at solution {solution_idx}, chunk {chunk_idx} with value {val_value.item()}")
                valid_chunks = chunks[:chunk_idx]  # Collect chunks up to this point
                break
            else:
            # If no value < 0.9 was found, consider all chunks
                valid_chunks = chunks

        # Concatenate valid chunks into a single string
        result_string = ''.join(valid_chunks)
        all_results.append(result_string)  # Add to the results list

        return all_results

def new_prompt(chunks):
        
        print(len(chunks))
        prompt=""
        for idx,value in enumerate(chunks) :
            val_values=validation_value(value)
            
            if val_values.item()>0.9:
                prompt=prompt+value
                
            else: 
                break
                
        return prompt
            

# Gradio interface for user input and output
def gradio_interface(question, correct_answer):
    iterations=5
    final_prediction, all_predictions,final_answer,freq = majority_vote(question, iterations)
    
    iteration_no=0
    freq=5
    while freq < 3:
        print(f"looped {iteration_no}")
        if iteration_no <3 :
            break
        else :
            iteration_no=iteration_no+1
        val_values_2d = []
        
        #for single feedback
        chunks = split_problem(final_prediction)
        newprompt = new_prompt(chunks)

        # Loop over all solutions in all_predictions
#         for solution in all_predictions:
#             # Split the solution into chunks
#             print(solution)
#             chunks = split_problem(solution)
#             newprompt = new_prompt(chunks)

        
        
        # Recalculate final_prediction and freq with the new prompt
        final_prediction, all_predictions, final_answer, freq = majority_vote(newprompt, iterations)

    # Once freq >= 3, return the results
    return {
        "Question": question,
        "Generated Answers (10 iterations)": all_predictions,
        "Majority-Voted Prediction": final_prediction,
        "Correct solution": correct_answer,
        "Majority answer": final_answer
    }
    



    


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
answer=gradio_interface("what is area of circle with radius lcm of 4 and 5?",4)
print(answer)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

frequency is  5
{'Question': 'what is area of circle with radius lcm of 4 and 5?', 'Generated Answers (10 iterations)': ["Solve the following mathematical problem: what is area of circle with radius lcm of 4 and 5?\n### Solution: To solve the problem of finding the area of a circle with a radius equal to the least common multiple (LCM) of 4 and 5, we can follow these steps:\n\n1. Calculate the LCM of 4 and 5.\n2. Use the radius obtained from the LCM to calculate the area of the circle.\n\nThe formula for the area of a circle is given by:\n\\[ \\text{Area} = \\pi \\times r^2 \\]\nwhere \\( r \\) is the radius of the circle.\n\nLet's implement this in Python using the sympy library.\n\n```python\nimport sympy as sp\n\n# Define the numbers\na = 4\nb = 5\n\n# Calculate the LCM of a and b\nlcm = sp.lcm(a, b)\n\n# Define the radius\nradius = lcm\n\n# Calculate the area of the circle\narea = sp.pi * radius**2\n\n# Print the area\nprint(area)\n```\n```output\n100*pi\n```\nThe area of the circl

In [42]:
answer=gradio_interface("what is area of circle with radius lcm of 4 and 5?",4)
print(answer["Majority-Voted Prediction"])
print("final answer: ",answer[Majority answer])

Solve the following mathematical problem: what is area of circle with radius lcm of 4 and 5?
### Solution: To solve the problem of finding the area of a circle with a radius equal to the least common multiple (LCM) of 4 and 5, we can follow these steps:

1. Calculate the LCM of 4 and 5.
2. Use the radius obtained from the LCM to calculate the area of the circle.

The formula for the area of a circle is given by:
\[ \text{Area} = \pi \times r^2 \]
where \( r \) is the radius of the circle.

Let's implement this in Python using the sympy library.

```python
import sympy as sp

# Define the numbers
a = 4
b = 5

# Calculate the LCM of a and b
lcm = sp.lcm(a, b)

# Define the radius
radius = lcm

# Calculate the area of the circle
area = sp.pi * radius**2

# Print the area
print(area)
```
```output
100*pi
```
The area of the circle with a radius equal to the least common multiple (LCM) of 4 and 5 is \(\boxed{100\pi}\).

final answer: 


In [43]:
text = """
Solve the following mathematical problem: what is area of circle with radius lcm of 4 and 5?
### Solution: To solve the problem of finding the area of a circle with a radius equal to the least common multiple (LCM) of 4 and 5, we can follow these steps:

1. Calculate the LCM of 4 and 5.
2. Use the radius obtained from the LCM to calculate the area of the circle.

The formula for the area of a circle is given by:
\[ \text{Area} = \pi \times r^2 \]
where \( r \) is the radius of the circle.

Let's implement this in Python using the sympy library.

```python
import sympy as sp

# Define the numbers
a = 4
b = 5

# Calculate the LCM of a and b
lcm = sp.lcm(a, b)

# Define the radius
radius = lcm

# Calculate the area of the circle
area = sp.pi * radius**2

# Print the area
print(area)
```
```output
100*pi
```
The area of the circle with a radius equal to the least common multiple (LCM) of 4 and 5 is \(\boxed{100\pi}\).
"""
chunks=split_problem(problem2)
for chunk in chunks:
    print(chunk)
    print("-------------------")
val_values = [[torch.tensor([[0.9450]], device='cuda:0'), 
               torch.tensor([[0.9400]], device='cuda:0'), 
               torch.tensor([[0.9416]], device='cuda:0'), 
               torch.tensor([[0.9537]], device='cuda:0')]]
new_input=new_input(chunks,val_values[0])

### Solution: To solve this problem, let's define the variables and set up the equations based on the given information.
-------------------
1. Let \( n \) be the number of employees 1.2 A.

-------------------
2. The manager initially planned to distribute $50 to each employee, so the total amount needed was \( 50n \). However, the fund was $5 less than what was needed, so the initial fund amount was \( 50n - 5 \).

-------------------
3. Instead, the manager distributed $45 to each employee, which amounted to \( 45n \), and kept the remaining $95 in the fund. Therefore, the initial fund amount was \( 45n + 95 \).
-------------------
We can set up the following equation:
\[ 50n - 5 = 45n + 95 \]

Let's solve this equation using Python and sympy.

```python
import sympy as sp

# Define the variable
n = sp.symbols('n')

# Set up the equation
equation = sp.Eq(50*n - 5, 45*n + 95)

# Solve the equation for n
solution = sp.solve(equation, n)

# Calculate the initial fund amount
initial_fun

NameError: name 'all_results' is not defined

In [ ]:

def new_prompt(chunks):
        
        print(len(chunks))
        prompt=""
        for idx,value in enumerate(chunks) :
            val_values=validation_value(value)
            
            if val_values.item()>0.9:
                prompt=prompt+value
                
            else: 
                break
                
        return prompt
            
                
            
            
          

new_input=new_prompt(chunks) 
new_input

## benchmarking

In [12]:
from datasets import load_dataset

dataset = load_dataset("lighteval/MATH", "all", split='test')
questions = [item['problem'] for item in dataset]
true_answers = [item['solution'] for item in dataset]


0000.parquet:   0%|          | 0.00/2.99M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [18]:
!pip install latex2sympy2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 5.5 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.7.2-py3-none-any.whl size=140931 sha256=8282d2e2d5869b1e675f0a8c479383dc1720b671c7213b522eae6784e69a9533
  Stored in directory: /root/.cache/pip/wheels/79/20/ec/30bf7dabc29319ccc0d0c96f910a640513a3c81faa960fed43
Successfully built antlr4-python3-runtime


In [44]:
from latex2sympy2 import latex2sympy
import re
from tqdm import tqdm

# Function to extract boxed values from LaTeX answers
def extract_boxed_value(answer):
    # Regular expression to match \boxed{...}
    match = re.search(r'\\boxed\{([^}]*)\}', answer)
    
    if match:
        # Extract the matched value inside the \boxed{...}
        extracted_value = match.group(1)
        
       
        if not extracted_value.endswith('}'):
            extracted_value += '}' 
        
        return extracted_value
    
    return None

# Initialize lists to store questions, true answers, and predicted answers
questions = [item['problem'] for item in dataset]
true_answers = [item['solution'] for item in dataset]
predicted_answers = []
accurate = 0  # Track the number of correct predictions

# Initialize tqdm progress bar
progress_bar = tqdm(total=len(questions), desc="Processing", ncols=100, unit="question")

for idx, question in enumerate(questions):
    # Get the answer from the Gradio interface
    answer = gradio_interface(question, 4)

    # Extract the actual answer and convert it to a symbolic expression using latex2sympy
    actual_answer = extract_boxed_value(true_answers[idx])
    if actual_answer:
        actual_answer = latex2sympy(actual_answer)
    
    # Check if the predicted answer from the Gradio interface is valid
    if answer["Majority answer"][1] is True:
        pred_answer = answer["Majority answer"][0]
    else:
        # If not, extract the boxed value from the true answer as the prediction
        pred_answer = extract_boxed_value(answer["Majority-Voted Prediction"])
    
#     # Convert the predicted answer to a symbolic expression
#     if pred_answer:
#         pred_answer = latex2sympy(pred_answer)
    print(pred_answer,actual_answer)
    
    # Compare predicted answer with actual answer
    if pred_answer == actual_answer:
        accurate += 1  # Increment if the answer is correct
    
    # Store the predicted answer
    predicted_answers.append(pred_answer)

    # Calculate accuracy so far
    accuracy = (accurate / (idx + 1)) * 100

    # Update progress bar and print the accuracy for this iteration
    progress_bar.update(1)
    progress_bar.set_postfix_str(f"Accuracy: {accuracy:.2f}%")
    
# Close the progress bar
progress_bar.close()

print(f"Final Accuracy: {accuracy:.2f}%")


Processing:   0%|                      | 2/5000 [19:21<806:06:19, 580.63s/question, Accuracy: 0.00%]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Expli

frequency is  5
(2.0, -3.0) 2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

frequency is  5
10.0 10


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

frequency is  5


Exception: I expected something else here
\frac{9}
~~~~~~~~^

In [33]:
print(answer["Majority answer"][0])

Solve the following mathematical problem: Find $x$ such that $\lceil x \rceil + x = \dfrac{23}{7}$. Express $x$ as a common fraction.
### Solution: To solve the problem, we need to find \( x \) such that \(\lceil x \rceil + x = \frac{23}{7}\). Here, \(\lceil x \rceil\) denotes the ceiling of \( x \), which is the smallest integer greater than or equal to \( x \).

Let's break down the problem into manageable steps:

1. **Understand the Ceiling Function:**
   The ceiling function \(\lceil x \rceil\) means that \( n \leq x < n+1 \) for some integer \( n \).

2. **Set Up the Equation:**
   Given \(\lceil x \rceil + x = \frac{23}{7}\), let's denote \(\lceil x \rceil\) as \( n \). Therefore, we have:
   \[
   n + x = \frac{23}{7}
   \]
   Since \( n \leq x < n+1 \), we can write:
   \[
   x = \frac{23}{7} - n
   \]
   and
   \[
   n \leq \frac{23}{7} - n < n+1
   \]

3. **Solve the Inequality:**
   We need to solve the inequality:
   \[
   n \leq \frac{23}{7} - n < n+1
   \]

Let's implemen